<img src="steak.png">


<b><font size="40">Steak A.I.</font></b>

Van Arloe M. Yu

# Abstract

With the global food industry expected to reach USD 12.24 trillion [1] and mobile sales to reach 2.05 billion units [2] in 2020, there is a big business opportunity to innovate on new ways to capitalize on this expanding market. Through a USD 1 app that generates recipes using artificial intelligence with almost zero operating cost, we can get a revenue of USD 200 million by just getting a 10\% share of the new mobile devices. In this notebook, I demonstrated how to generate steak recipes using generative deep learning using textgenrnn which is a recurrent neural network model. I trained model using 444 steak recipes gathered online. Through sensitivity analysis, I was able to arrive at a model that has a cross entropy loss of 0.422 compared to the default baseline of 0.590. Additionaly, through sensitivity analysis, I was able to select 0.5 as the best temperature parameter to obtain realistic steak recipes.

# Introduction

The global food and grocery retail market size is expected to reach USD 12.24 trillion by 2020, according to a new report by Grand View Research, Inc. The industry has been growing steadily in recent years and the trend shows that it will even continue to grow. Countries from all over the world are experiencing the growth in food. The rise of technology and economies have been playing a pivotal role in the growth of the food industry. The modern lifestyle that is looking for quick and easy ways to prepare food while still highlighting quality food demands innovators to supply new ways to tap the market. [1]

In 2020, mobile sales volume is expected to reach 2.05 billion units. A USD 1 app that generates recipes using artificial intelligence with almost zero operating cost, we can get a revenue of USD 200 million. [2]

In this paper, I applied generative deep learning using textgenrnn to generate steak recipes. Textgenrnn is a recurrent neural network based model. It is based off of the char-rnn project by Andrej Karpathy and uses Attention-layer code used from DeepMoji (MIT Licensed). [3]

# Data Description

The data came from an Dave Shield's collection of recipes. I only used the steak recipes in particular for this notebook. He described the recipes to be in the "Now You're Cooking Format." The data is in the text file format. The text file consists of recipes that conserved the pattern as given below. [4]

The data consists of a Now You're Cooking header, the recipe name, tags, ingredient quantity and description, cooking instructions, and serving size. Note that each section is separated by a line space. A sample of one recipe can be seen below.

    ** Exported from Now You're Cooking! v5.71 **

    @@@@@ Now You're Cooking! Export Format

    Grilled Flank Steak With Chimichurri Sauce

    meats, vegetables, ethnic

    2 lb flank steak
          ----SAUCE----
    4  jalapeno peppers, remove stems and; seeds, chop fine
    1/2 cup chopped onion
    4  cloves garlic, minced
    1/4 cup chopped fresh parsley
    1 teaspoon oregano
    1 teaspoon fresh ground black pepper
    1/2 cup olive oil
    1/4 cup red wine vinegar
    2 tablespoon lemon juice
    1/4 cup water

    Combine all ingredients for the Chimichuri Sauce.
    Score each side of the steak about 1/8 to 1/4" deep and rub in the
    sauce, reserving some of the sauce to serve with the meal. Place the
    meat and marinade in a ceramic or glass pan and marinate overnight in
    the refrigerator. Remove the meat and extra sauce from the
    refigerator and allow to sit at room temperature before grilling.
    Grill the meat over charcoal or under a broiler until medium-rare.
    Carve the steak immediately accross the grain into strips about 1/4"
    thick and serve with extra sauce. If the meat is allowed to sit
    before carving, it will toughen.

    Yield: 6 servings 
    


Taken from: http://www.shieldsgenealogy.com/recipe-nyc-dl.html


# Methodology

Recurrent neural networks (RNNs) have been applied in sequential data such as text, audio, and video. It is a class of artificial neural networks that can update the current state based on states and current input data. In this implementation I used the textgenrnn which uses char rnn at its core and DeepMoji's attention layer. I trained the textgenrnn model using the steak recipes discussed in the data description. I used four different sets of parameters which I will discuss in the implementation section of below. 

## char-rnn

"char-rnn" stands for for “character recurrent neural network”. It is a recurrent neural network that predicts the next character given a sequence of characters. It implements multi-layer Recurrent Neural Network (RNN, LSTM, and GRU) for training/sampling from character-level language models. Since it is a recurrent neural network, the predicted output becomes the input for the next iteration and so on. This can be visualized by looking at the example below wherein the whole phrase is "Hello darkness my old friend". Frome here we see from the first row that y is the predicted character. In the succeeding row, the y becomes one of the input after the window is moved by one character. [5]

<img src="darkness.png"> 
Taken from: https://hjweide.github.io/char-rnn

It is essentially a classification model that output a probability distribution over the classes. It takes the character with the highest probability as its prediction of the next character of the text sequence. [5]

Since the output of the model is the probabilities for classification, the standard categorical cross-entropy loss to train the model. Given a vector $y$ as the one-hot encoded vector representation of the target character and $\hat{y}$  as the probabilities over all n characters considered in the output layer vocabulary, the loss function is given as:
$$\sum_{i=1}^{n} \ln{y_i}\log{\hat{y}}$$

By training the model iteratively, our model learns to predict characters consistent with the patterns sequences observed during training. This enables the model to capture the overall style of the author. It is intuitive that a consistent dataset would be beneficial for our model to learn with a higher accuracy and shorting training time. [5]



## textgenrnn

Textgenrnn's maintainer and creator is Max Woolf. It is based off of the char-rnn project by Andrej Karpathy and uses Attention-layer code used from DeepMoji (MIT Licensed). [3]

<img src="textgenrnn.png">
Taken from: https://github.com/minimaxir/textgenrnn

The pretrained-model follows a neural network architecture inspired by DeepMoji as described the the creator. The model takes an input of 40 characters by defaults, but can be tuned in the parameters. Each character is converted to a 100-D character embedding vector, a technique to conserve the meaning and context. It is then fed into a 128-cell long-short-term-memory (LSTM) recurrent layer and then the output is inputed into another 128-cell long-short-term-memory (LSTM) recurrent layer. The three layers are then fed into an Attention layer that was taken from DeepMoji to weigh the most important temporal features and average them together. The skip connection going into the attention layer was design to backpropagate easily and avoid vanishing gradients. Finally the output is mapped to the probabilities of 394 unique characters. These 394 characters include uppercase characters, lowercase, punctuation, and emoji. [3]

The textgenrnn also has a feature for contextual learning which accepts a categorical tag for every text sequence such that different sequences can be tagged differently and the model would learn the text sequences separately. This can be used in application such as recipe generation where you will tag pork, chicken, beef, soup, or desserts separately so that the model will not mix up recipes of desserts and pork together. [3]

## Implementation

This implementation is based on the free interactive textgenrnn Demo w/ GPU by Max Woolf at Google Colab. [6] For the implementation, the baseline would be the parameters in the demo notebook. I also explored adjusting the parameters to do a sensitivity analysis on how the results would change. First, I trained a model using a bidirectional LSTM that considers the sequence in both backward and forward directions. Second, I trained a model with an additional dropout layer of 10%. This would help make the model more robust. Third, I trained a model that learns at a word level instead of a character level. 

In [0]:
!pip install -q textgenrnn
from textgenrnn import textgenrnn
from google.colab import files
from datetime import datetime
import tensorflow
import os

In [0]:
tensorflow.__version__

'1.15.0'

Please note that the model only runs on tensorflow version 1.15.0, it has compatibility issues with tensorflow version 2.0.0 and above.

In [0]:
file_name = "steak.txt"

### Default Model

In [0]:
model_name = 'default'   # change to set file name of resulting trained models/texts
model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 3,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 40,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 20,   # set higher to train the model for longer
    'gen_epochs': 10,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.0,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [18]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

Training new model w/ 3-layer, 128-cell LSTMs
Training on 370,733 character sequences.
Epoch 1/20
362/362 [==============================] - 38s 105ms/step - loss: 3.0707
Epoch 2/20
362/362 [==============================] - 36s 98ms/step - loss: 1.8618
Epoch 3/20
362/362 [==============================] - 36s 100ms/step - loss: 1.4186
Epoch 4/20
362/362 [==============================] - 36s 100ms/step - loss: 1.2291
Epoch 5/20
362/362 [==============================] - 36s 100ms/step - loss: 1.1147
Epoch 6/20
362/362 [==============================] - 37s 101ms/step - loss: 1.0427
Epoch 7/20
362/362 [==============================] - 37s 101ms/step - loss: 0.9830
Epoch 8/20
362/362 [==============================] - 37s 101ms/step - loss: 0.9355
Epoch 9/20
362/362 [==============================] - 38s 105ms/step - loss: 0.8912
Epoch 10/20
362/362 [==============================] - 37s 103ms/step - loss: 0.8533
####################
Temperature: 0.2
####################
sides over mea

In [ ]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

The following cell is used to generated text using the trained model.

In [0]:
# this temperature schedule cycles between 1 very unexpected token, 1 unexpected token, 2 expected tokens, repeat.
# changing the temperature schedule can result in wildly different output!
temperature = [1.0, 0.5, 0.2, 0.2]   
prefix = None   # if you want each generated text to start with a given seed text

if train_cfg['line_delimited']:
    n = 1000
    max_gen_length = 60 if model_cfg['word_level'] else 300
else:
    n = 1
    max_gen_length = 2000 if model_cfg['word_level'] else 10000

timestring = datetime.now().strftime('%Y%m%d_%H%M%S')
gen_file = '{}_gentext_{}.txt'.format(model_name, timestring)

textgen.generate_to_file(gen_file,
                         temperature=temperature,
                         prefix=prefix,
                         n=n,
                         max_gen_length=max_gen_length)
files.download(gen_file)

The following cell is used to download the weights, vocabulary, and configuration of the model trained. You can use this to save your model.

In [0]:
files.download('{}_weights.hdf5'.format(model_name))
files.download('{}_vocab.json'.format(model_name))
files.download('{}_config.json'.format(model_name))

### Bidirectional LSTM

In [ ]:
model_name = 'bidirectional'   # change to set file name of resulting trained models/texts

model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 3,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': True,   # consider text both forwards and backward, can give a training boost
    'max_length': 40,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 20,   # set higher to train the model for longer
    'gen_epochs': 10,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.0,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [9]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

Training new model w/ 3-layer, 128-cell Bidirectional LSTMs
Training on 371,204 character sequences.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
362/362 [==============================] - 39s 107ms/step - loss: 2.7890
Epoch 2/20
362/362 [==============================] - 32s 87ms/step - loss: 1.9346
Epoch 3/20
362/362 [==============================] - 32s 87ms/step - loss: 1.4248
Epoch 4/20
362/362 [==============================] - 32s 87ms/step - loss: 1.1815
Epoch 5/20
362/362 [==============================] - 32s 88ms/step - loss: 1.0516
Epoch 6/20
362/362 [==============================] - 32s 88ms/step - loss: 0.9629
Epoch 7/20
362/362 [==============================] - 32s 88ms/step - loss: 0.8944
Epoch 8/20
362/362 [==============================] - 32s 88ms/step - loss: 0.8382
Epoch 9/20
362/362 [==============================] - 32s 89ms/step - loss: 0.7885
Epoch 10/20
362/362 [==============================] - 

The following cell is used to generated text using the trained model.

In [0]:
# this temperature schedule cycles between 1 very unexpected token, 1 unexpected token, 2 expected tokens, repeat.
# changing the temperature schedule can result in wildly different output!
temperature = [1.0, 0.5, 0.2, 0.2]   
prefix = None   # if you want each generated text to start with a given seed text

if train_cfg['line_delimited']:
    n = 1000
    max_gen_length = 60 if model_cfg['word_level'] else 300
else:
    n = 1
    max_gen_length = 2000 if model_cfg['word_level'] else 10000

timestring = datetime.now().strftime('%Y%m%d_%H%M%S')
gen_file = '{}_gentext_{}.txt'.format(model_name, timestring)

textgen.generate_to_file(gen_file,
                         temperature=temperature,
                         prefix=prefix,
                         n=n,
                         max_gen_length=max_gen_length)
files.download(gen_file)

The following cell is used to download the weights, vocabulary, and configuration of the model trained. You can use this to save your model.

In [0]:
files.download('{}_weights.hdf5'.format(model_name))
files.download('{}_vocab.json'.format(model_name))
files.download('{}_config.json'.format(model_name))

### Dropout 10% Layer

In [ ]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])
model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 3,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 40,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 20,   # set higher to train the model for longer
    'gen_epochs': 10,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.1,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [8]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])




Training new model w/ 3-layer, 128-cell LSTMs

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Training on 371,540 character sequences.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
362/362 [==============================] - 28s 79ms/step - loss: 3.3033
Epoch 2/20
362/362 [==============================] - 23s 62ms/step - loss: 3.0236
Epoch 3/20
362/362 [==============================] - 23s 63ms/step - loss: 1.9470
Epoch 4/20
362/362 [==============================] - 23s 63ms/step - loss: 1.4709
Epoch 5/20
362/362 [==============================] - 23s 63ms/step - loss: 1.2975
Epoch 6/20
362/362 [==============================] - 23s 64ms/step - loss: 1.1773
Epoch 7/20
362/362 [==============================] - 23s 64ms/step - loss: 1.0941
Epoch 8/20
362/362 [==============================] - 23s 63ms/step - loss: 1.0370
Epoch 9/20
362/362 [==============

The following cell is used to generated text using the trained model.

In [0]:
# this temperature schedule cycles between 1 very unexpected token, 1 unexpected token, 2 expected tokens, repeat.
# changing the temperature schedule can result in wildly different output!
temperature = [1.0, 0.5, 0.2, 0.2]   
prefix = None   # if you want each generated text to start with a given seed text

if train_cfg['line_delimited']:
    n = 1000
    max_gen_length = 60 if model_cfg['word_level'] else 300
else:
    n = 1
    max_gen_length = 2000 if model_cfg['word_level'] else 10000

timestring = datetime.now().strftime('%Y%m%d_%H%M%S')
gen_file = '{}_gentext_{}.txt'.format(model_name, timestring)

textgen.generate_to_file(gen_file,
                         temperature=temperature,
                         prefix=prefix,
                         n=n,
                         max_gen_length=max_gen_length)
files.download(gen_file)

The following cell is used to download the weights, vocabulary, and configuration of the model trained. You can use this to save your model.

In [0]:
files.download('{}_weights.hdf5'.format(model_name))
files.download('{}_vocab.json'.format(model_name))
files.download('{}_config.json'.format(model_name))

### Word Level

In [ ]:
model_name = 'word_level'   # change to set file name of resulting trained models/texts

model_cfg = {
    'word_level': True,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 3,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 40,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 20,   # set higher to train the model for longer
    'gen_epochs': 10,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.0,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [12]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])












Training new model w/ 3-layer, 128-cell LSTMs
Training on 97,280 word sequences.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
95/95 [==============================] - 15s 154ms/step - loss: 5.2683
Epoch 2/20
95/95 [==============================] - 9s 92ms/step - loss: 4.5610
Epoch 3/20
95/95 [==============================] - 9s 90ms/step - loss: 4.2357
Epoch 4/20
95/95 [==============================] - 9s 92ms/step - loss: 3.6747
Epoch 5/20
95/95 [==============================] - 9s 92ms/step - loss: 3.2084
Epoch 6/20
95/95 [==============================] - 9s 91ms/step - loss: 2.8759
Epoch 7/20
95/95 [==============================] - 8s 88ms/step - loss: 2.6159
Epoch 8/20
95/95 [==============================] - 9s 93ms/step - loss: 2.3936
Epoch 9/20
95/95 [==============================] - 9s 92ms/step - loss: 2.2211
Epoch 10/20
95/95 [==============================] - 9s 91ms/step - loss: 2.0486
#########

The following cell is used to generated text using the trained model.

In [0]:
# this temperature schedule cycles between 1 very unexpected token, 1 unexpected token, 2 expected tokens, repeat.
# changing the temperature schedule can result in wildly different output!
temperature = [1.0, 0.5, 0.2, 0.2]   
prefix = None   # if you want each generated text to start with a given seed text

if train_cfg['line_delimited']:
    n = 1000
    max_gen_length = 60 if model_cfg['word_level'] else 300
else:
    n = 1
    max_gen_length = 2000 if model_cfg['word_level'] else 10000

timestring = datetime.now().strftime('%Y%m%d_%H%M%S')
gen_file = '{}_gentext_{}.txt'.format(model_name, timestring)

textgen.generate_to_file(gen_file,
                         temperature=temperature,
                         prefix=prefix,
                         n=n,
                         max_gen_length=max_gen_length)
files.download(gen_file)

The following cell is used to download the weights, vocabulary, and configuration of the model trained. You can use this to save your model.

In [0]:
files.download('{}_weights.hdf5'.format(model_name))
files.download('{}_vocab.json'.format(model_name))
files.download('{}_config.json'.format(model_name))

# Results and Analysis

| Parameters    | Loss   |
|---------------|--------|
| Default       | 0.5898 |
| Bidirectional | 0.4220 |
| 10% Dropout   | 0.7181 |
| Word Level    | 1.1528 |

Comparing the results of the four sets of parameters, we can see that the bidirectional LSTM achieved the best result. This is intuitive because here the the output layer can get information from in both the past and future states. Although it may not give the best results for all sequence problems, it provides good results for some text sequences which makes it a choice for application like speech recognition and translation. 

After training our model, we can now generate steak recipes. When generating recipes, the generated text can be tuned by the parameter temperature. Temperature represents the creativity or conservativeness of the text generated. The higher the temperature is, the more creative the output is and less conservative. Temperature lowers the threshold at which characters are activated, thus resulting to more diverse results but at the cost of more mistakes. A low temperature is more conservative and has less mistakes but may fall into a trap of repeating predictions.

The default temperature for textgenrnn is 0.5. Here we tried to see what the output looks like for temperatures 0.2, 0.5, and 1.0. 

In [4]:
from textgenrnn import textgenrnn
textgen = textgenrnn(weights_path='bidirectional_weights.hdf5',
                       vocab_path='bidirectional_vocab.json',
                       config_path='bidirectional_config.json')
         
textgen.generate_samples(max_gen_length=1000)
textgen.generate_to_file('textgenrnn_texts.txt', max_gen_length=1000)

####################
Temperature: 0.2
####################
n with salt and pepper.  Heat oil in a large skillet over medium heat until hot. Add beef and cook until thickened, for 1 minute. Stir in the steaks are tender. Sprinkle with parsley.

Serves 4.

Yield: 4 servings

  
  
  

** Exported from Now You're Cooking! v5.71 **

@@@@@ Now You're Cooking! Export Format

Steak & Kidney Sponge

meats, sauces

1 lb beef top sirloin steak, 1 thick
1  (1 1/2 lb. thick
1  steak sauce
1 tablespoon worcestershire sauce
1/2 teaspoon salt
1/4 teaspoon pepper
1 teaspoon salt
1/4 teaspoon pepper
1 teaspoon salt
1/2 teaspoon pepper
1 teaspoon salt
1/2 teaspoon pepper
1 teaspoon salt
1/2 teaspoon pepper
1 teaspoon salt
1/2 teaspoon pepper
1 teaspoon ground cumin
1/4 teaspoon garlic salt
1/2 teaspoon pepper
1 teaspoon salt
1/4 teaspoon pepper
1 teaspoon salt
1/2 teaspoon pepper
1 teaspoon dried oregano
1 teaspoon dried oregano
1/4 cup chopped onion
1/4 cup chopped onion
1 cup sugar
1 teaspoon salt
1/4

er, on both sides
with skile and
chopped prepared with a little
of plastic bag. Pour off the hein sauce to at the egg, thren, stirring constantly.

From Fat Genue Flank Steak

beef

1 each nest fresh steaks
2 tablespoon oil
2 tablespoon olive oil
1 can yoghtly leaves, removed
2 tablespoon vegder
1 cup tomato juice
1/4 cup chopped curry powder & celery

1.  Fat zumin beef and red-hoppings,
bell with mashed potatoes or the piecrust and refrigerate at least 30 modium rare. Remove
loose at
rarve steaks.Toss steak for 1 to 2 minutes or until the meat is slow
cooker. Roll broiler.  Sprinkle with salt. Spoon sauce: Sprinkle with a small amount of your guide until nicely a liking.

Roy Linda Fiest, TX
Yizzed by: Slumb Steak Wellet Flank steak, Oring

main dish, meats, beef, marinades

1 1/2 lb beef round steak\,
1  onion round
3 fabout round steak
1 medium garlic clove -- huce and chopped
parsley
1  french c-rob, fine dry septhion

Combine first in toast, so etweel to the done. Stir in bacon
s

We see that for temperature 0.2, there are a lot of repeating ingredients. There is no drastic mistake on how it defines the recipe, but it is too safe in predicting the next ingredient by just sticking to common ingredients like salt.

We see that for temperature 1.0, there are less repeating ingredients. The model looks like it is more creative in its output by not being trapped to just common ingredients. However, we can see that there are typographical-error-like mistakes in spelling. This is because a temperature of 1.0 is best for well trained models that have been trained to thousands of data points.

Finally, when we look at the default temperature of 0.5, we are able to achieve a balance between creativity and conservativeness. Looking at the output, we have less repeating ingredients and more creative predictions. We also minimize the errors in spelling.

# Conclusion and Recommendations

In this notebook, I demonstrated how to generate steak recipes using generative deep learning using textgenrnn which is a recurrent neural network model. I trained model using 444 steak recipes gathered online. 

Through sensitivity analysis, I was able to arrive at a model that has a loss of 0.422 compared to the default baseline of 0.590. The final model selected uses a bidirectional LSTM compared to the default regualr LSTM model. 

Additionaly, through sensitivity analysis, I was able to select 0.5 as the best temperature parameter to obtain realistic steak recipes. This temperature achieves a balance between creativity and conservativeness. It generates recipes that are unique with minimal repeating ingredients and minimal spelling errors.

From our results we can see that the generated text would not be of high quality 100% of the time. According to the creator, this is why in practice the generated text is usually filtered, curated, and edited before publishing afterwards. 

We can also say that the results vary greatly between datasets. The patters between different corpus would vary greatly, and the application of the model to different data would not be the same. If the dataset is small, thus more training is required. Currently there is no rule or establish heuristics on training to achieve the best results. However, the creator recommends least 2,000-5,000 documents to train a good model, where in our application we were only able to use 444 recipes or documents.

In extending the study, the contextual mode can be used to develop more recipes that can be based on types of meat or cuisine. More and more products today target niche markets. With the contextual mode, we can train focused on a certain niche that can be marketed to target audience.

# Acknowledgements

I would like to acknowledge our professors for the course Machine Learning 2.0 at the Asian Institute of Management, Prof. Chris Monterola and Prof. Erika Legara. I would also like to thank Google Colab and Acer for providing me with the computing resources for this project.

I would like to dedicate this quote to aspiring data scientists who are still exploring all the possibilities of deep learning:
    
    For the things we have to learn before we can do them, we learn by doing them.
    - Aristotle

# References

    [1] The global food and grocery retail market size is expected to reach USD 12.24 trillion by 2020. (2018, August 27). Retrieved from https://www.prnewswire.com/news-releases/the-global-food-and-grocery-retail-market-size-is-expected-to-reach-usd-12-24-trillion-by-2020--300702659.html.
    [2] Mobile sales volume projected to reach 2.05B by 2020. (n.d.). Retrieved from https://www.retaildive.com/ex/mobilecommercedaily/mobile-sales-volume-projected-to-reach-2-05b-by-2020-report.
    [3] Textgenrnn. Retrieved from https://github.com/minimaxir/textgenrnn.
    [4] Recipes that I've collected. Retrieved from http://www.shieldsgenealogy.com/recipe-nyc-dl.html.
    [5] Training a char-rnn to Talk Like Me. Retrieved from https://hjweide.github.io/char-rnn.
    [6] Interactive textgenrnn Demo w/ GPU. Retrieved from https://drive.google.com/file/d/1mMKGnVxirJnqDViH7BDJxFqWrsXlPSoK/view.